for more scope creep, using a 30 (or 60 or 90) day price plot estimate the most likley day that the stock will peak and use this as the sell date.


In [1]:
import pandas as pd
from datetime import datetime, timedelta, time
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
#from alpha_vantage.techindicators import TechIndicators
from pandas.tseries.offsets import BMonthEnd
#from dotenv import load_dotenv
import os
import pickle
from ta.momentum import RSIIndicator
from ta.trend import MACD
#import tensorflow as tf
import itertools
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix # Loading required libraries



In [ ]:
print('hi')

In [2]:
df = pd.read_csv('2015_2025_data/insider_trades.csv', parse_dates=['transaction_date', 'trade_date'])
df = df.dropna()

for i in ['last_price', 'Qty', 'shares_held', 'Owned', 'Value']:
    df[i] = (
        df[i]
        .astype(str)
        .str.replace('New', '1000000000', regex=False)
        .str.replace('[\$,\+\-\%>]', '', regex=True)
        .astype(float)
    )

df['transaction_date'] = pd.to_datetime(df['transaction_date'], format="%d/%m/%Y %H:%M")
df['trade_date'] = pd.to_datetime(df['trade_date'])

#print(type(df['transaction_date']))
#print(df['transaction_date'])
#print(df['trade_date'])

df['Owned_norm'] = 1 - np.exp(-df['Owned']*np.log(2)/100)

df = df.sort_values(by='transaction_date')

#df = df.loc[:499]


<>:9: SyntaxWarning: "\$" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\$"? A raw string is also an option.
<>:9: SyntaxWarning: "\$" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\$"? A raw string is also an option.
C:\Users\azzag\AppData\Local\Temp\ipykernel_2656\1495930421.py:9: SyntaxWarning: "\$" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\$"? A raw string is also an option.
  .str.replace('[\$,\+\-\%>]', '', regex=True)


### Saving and loading varilabes with pickle

In [ ]:
#os.makedirs('saved_vars', exist_ok=True)  # create folder if it doesn't exist

'''
Saving a variable
'''
#with open('saved_vars/prices_for_all_tickers.pkl', 'wb') as f:
#    pickle.dump(prices, f)



'''
Loading a variable
'''
#with open('saved_vars/500_trades_from_mid_2024.pkl', 'rb') as f:  # 'rb' = read binary
#    data = pickle.load(f)

'\nLoading a variable\n'

In [3]:
with open('saved_vars/all_ticker_2015_25.pkl', 'rb') as f:  # 'rb' = read binary
    data = pickle.load(f)
df_by_ticker = data 


#### Load in prices from saved_vars

In [4]:
with open('saved_vars/prices_for_all_tickers.pkl', 'rb') as f:  # 'rb' = read binary
    prices = pickle.load(f)


In [ ]:
sal = prices['SAL']



## Creating features

In [ ]:


def create_features(trade):
    """
    ticker_data: DataFrame with OHLCV for one stock over 3 months
    Returns: Single row of engineered features:
        '1mo_return', '3mo_return', '30d_volatility', 'rsi_14', 'macd',
        'volume_zscore', 'price_vs_sma50'.
    """
    ticker_data = trade['prev_prices']

    features = {}

    features['trade_date_epoch'] = pd.to_datetime(trade['trade_date']).timestamp() # seconds since epoch
    features['eff_trans_date_epoch'] = pd.to_datetime(trade['eff_trans_date']).timestamp() 

    
    # Price returns
    features['1mo_return'] = ticker_data['Close'].iloc[-1] / ticker_data['Close'].iloc[-22] - 1
    features['3mo_return'] = ticker_data['Close'].iloc[-1] / ticker_data['Close'].iloc[-63] - 1
    
    # Volatility
    features['30d_volatility'] = ticker_data['Close'].pct_change().std() * np.sqrt(252)
    
    # Momentum indicators
    # RSI 14
    rsi = RSIIndicator(close=ticker_data['Close'], window=14).rsi()  # series
    features['rsi_14'] = rsi.iloc[-1]
    # MACD line
    macd = MACD(close=ticker_data['Close'])
    features['macd'] = macd.macd().iloc[-1]  # MACD line
    
    # Volume
    features['volume_zscore'] = (
        (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean()) 
        / ticker_data['Volume'].std()
    )
    
    # Trend relationships
    features['price_vs_sma50'] = ticker_data['Close'].iloc[-1] / ticker_data['Close'].rolling(50).mean().iloc[-1]

    # Bool on whether the transaction was made within trading hours.
    features['is_during_market_hours'] = time(hour=9, minute=30) < trade['transaction_date'].time() < time(hour=16)

    # Day of the week the trade was made on.
    features['day_of_week'] = trade['trade_date'].dayofweek

    # Number of days between making the trade and filing with the SEC
    features['filing_lad_days'] = (trade['transaction_date']-trade['trade_date']).days

    #df_by_ticker = df_by_ticker.dropna()
    
    title_str = ''.join(trade['Title'])
    if 'CEO' in title_str:
        features['title_rank'] = 4
    elif 'C' in title_str:
        features['title_rank'] = 3
    elif 'Dir' in title_str:
        features['title_rank'] = 2
    else:
        features['title_rank'] = 1
        
    return pd.Series(features)


def flag_month_end(trade_dates):
    """Returns 1 for last 3 business days of month"""
    flags = []
    for date in trade_dates:
        month_end = BMonthEnd().rollforward(date)  # Get official month-end
        last_3_days = pd.bdate_range(end=month_end, periods=3)  # Last 3 trading days
        flags.append(1 if date in last_3_days else 0)
    return flags

features = df_by_ticker.apply(lambda trade: create_features(trade), axis=1)


# Looks at the last valid trading date on or before the transaction date.
def get_spy_return(effective_date, spy_history):
    """Returns SPY's 1-day return for the last valid trading day <= effective_date"""
    # Get all SPY dates <= effective_date
    valid_dates = spy_history.index[spy_history.index.date <= effective_date]
    
    if len(valid_dates) == 0:
        return np.nan  # No previous trading data
    
    # Take the closest trading date
    dt = valid_dates[-1]
    return (spy_history.loc[dt, 'Close'] / spy_history.loc[dt, 'Open']) - 1

df_by_ticker['SPY_1d_return'] = df_by_ticker['eff_trans_date'].apply(
    lambda x: get_spy_return(x, spy_history)
)


df_by_ticker[features.columns] = features
df_by_ticker['month_end_flag'] = flag_month_end(df_by_ticker['trade_date'])
df_by_ticker = df_by_ticker.dropna()


KeyError: 'prev_prices'

### Create target variable for price movement more than x%
#### True for increase, False for decrease.

In [ ]:
def create_targets(prices):
    targets = {}

    targets['next_day'] = 1 < prices['Close'].iloc[0]/prices['Open'].iloc[0]
    targets['one_week'] = 1 < prices['Close'].iloc[5]/prices['Open'].iloc[0]
    targets['one_month'] = 1 < prices['Close'].iloc[20]/prices['Open'].iloc[0] #weekends not included => 1 week = 5 elements

    targets['next_day_2%'] = 1.02 < prices['Close'].iloc[0]/prices['Open'].iloc[0]
    targets['one_week_2%'] = 1.02 < prices['Close'].iloc[5]/prices['Open'].iloc[0]
    targets['one_month_2%'] = 1.02 < prices['Close'].iloc[20]/prices['Open'].iloc[0]


    targets['next_day_5%'] = 1.05 < prices['Close'].iloc[0]/prices['Open'].iloc[0]
    targets['one_week_5%'] = 1.05 < prices['Close'].iloc[5]/prices['Open'].iloc[0]
    targets['one_month_5%'] = 1.05 < prices['Close'].iloc[20]/prices['Open'].iloc[0]


    return pd.Series(targets)





targets = df_by_ticker.apply(lambda trade: create_targets(trade['future_prices']), axis=1)

In [ ]:
df_by_ticker.columns

## prediction 1 day higher (1) lower (0) price movements 


### (think about making that higher or lower than a x% increase, whatever the cut off is)

1. Binomial Logistic Regression (Good Baseline): An issue with this is that not all variables will be linearly independant. Good to get as many finacial ratios and indicators leading up to the purchase. But still do real basic analysis.

2. Random Forest: better for nonlin relationships, machine learning supervised classification modele.

****** 3. XGBoost is best "XGBoost with well-engineered features outperforms both binomial regression and deep learning in production.", best with tabular data. Min samples 5k, ideal >50k sample.

4. LSTM, for time series data (price data leading up to the date of purchase.) clean data, GPU resources, >50,000 samples required



#### To be added
days_since_earnings

Sector_ETF_5d_trend

VIX_level

This could be good to put in as well only if the filing_lag_days has some significance:

```
df['urgent_filing'] = (df['filing_lag_days'] <= 1).astype(int)  # Binary flag
df['lag_x_volume'] = df['filing_lag_days'] * df['trade_volume']  # Interaction term
```

#### GPT bullhsit you could add but i can't be bothered rn

### **1. Insider-Specific Features**
1. **`insider_cluster_size`**  
   - Number of unique insiders trading the same stock within 5 days  
   ```python
   df['insider_cluster_size'] = df.groupby(['ticker', pd.Grouper(key='trade_date', freq='5D')])['owner_names'].transform('nunique')
   ```

2. **`title_rank`**  
   - Numeric hierarchy of insider titles (CEO=4, CFO=3, Director=2, Other=1)  
   ```python
   title_rank = {'CEO':4, 'CFO':3, 'Director':2}
   df['title_rank'] = df['Title'].map(title_rank).fillna(1)
   ```

---

### **2. Advanced Market Context**
3. **`sector_relative_strength`**  
   - Stock's 5-day return vs sector ETF (e.g., XLK for tech)  
   ```python
   df['sector_relative_strength'] = df['1mo_return'] - sector_etf_returns
   ```

4. **`vix_1d_change`**  
   - Daily % change in VIX (volatility spike indicator)  
   ```python
   df['vix_1d_change'] = vix_data['Close'].pct_change().loc[df['eff_trans_date']].values
   ```

---

### **3. Technical Enhancements**
5. **`atr_14`**  
   - Average True Range (volatility normalization)  
   ```python
   df['atr_14'] = talib.ATR(df['High'], df['Low'], df['Close'], 14)
   ```

6. **`obv_5d`**  
   - 5-day On-Balance Volume trend  
   ```python
   df['obv_5d'] = talib.OBV(df['Close'], df['Volume']).pct_change(5)
   ```

---

### **4. Behavioral Signals**
7. **`short_interest_ratio`**  
   - Short interest / float (squeeze potential)  
   ```python
   df['short_interest_ratio'] = df['ticker'].map(short_interest_data)
   ```

8. **`earnings_proximity`**  
   - Days until next earnings (-30 to +30, 0=earnings day)  
   ```python
   df['earnings_proximity'] = (earnings_dates - df['trade_date']).dt.days
   ```

---

### **5. Risk Management**
9. **`beta_1mo`**  
   - Stock's 1-month beta to SPY  
   ```python
   df['beta_1mo'] = df['ticker'].map(beta_calculations)
   ```

10. **`liquidity_zscore`**  
    - Current volume vs 3-month avg (z-score)  
    ```python
    df['liquidity_zscore'] = (df['Volume'] - df['Volume'].rolling(63).mean()) / df['Volume'].rolling(63).std()
    ```


## Random Forest Classifier

In [ ]:
y = targets['next_day_2%']
X = df_by_ticker.drop(['transaction_date', 'trade_date', 'eff_trans_date', 'ticker','company_name', 'owner_names', 'prev_prices', 'future_prices', 'Title'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) # Spliting Train Test datasets

In [ ]:
clf=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_leaf_nodes=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=90,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [ ]:
print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

y_pred = clf.predict(X_test)
confusion_mtx = confusion_matrix(y_test, y_pred) 
plot_confusion_matrix(confusion_mtx, classes = range(2)) 
# Confusion Matrix 

## XGBoost